In [ ]:
# default_exp distanceApproaches

In [2]:
# export
from multiinstance.utils import *

import seaborn as sns

import community as community_louvain
import networkx as nx

from multiinstance.data.syntheticData import buildDataset,getBag
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KernelDensity
from scipy.special import logsumexp
import scipy.stats as ss
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'community'

In [ ]:
# export
def fitKDE(vec):
    kde = KernelDensity(kernel="gaussian").fit(vec)
    return kde

def KLD(lnDensI,lnDensJ):
        return ss.entropy(np.exp(lnDensI), qk=np.exp(lnDensJ),base=2)
    
def JSD(ds, kdeI, i, j):
    _,uI = getTransformScores(ds,i)
    uI = uI.reshape((-1,1))
    _,uJ = getTransformScores(ds,j)
    uJ = uJ.reshape((-1,1))
    kdeJ = fitKDE(uJ)
    lnDensI0 = kdeI.score_samples(uI)
    lnDensJ0 = kdeJ.score_samples(uI)
    lnDensM0 = np.array([logsumexp((ldi,ldj),
                                       b=np.array([.5,.5])) for ldi,ldj in zip(lnDensI0, lnDensJ0)])
    lnDensI1 = kdeI.score_samples(uJ)
    lnDensJ1 = kdeJ.score_samples(uJ)
    lnDensM1 = np.array([logsumexp((ldi,ldj),
                                       b=np.array([.5,.5])) for ldi,ldj in zip(lnDensI1, lnDensJ1)])
    x = KLD(lnDensI0,lnDensM0)
    y = KLD(lnDensJ1, lnDensM1)
    return x + y

def getJSDDistMat(ds):
    N = ds.N
    dist = np.zeros((N,N))
    for i in range(N):
        _, uI = getTransformScores(ds,i)
        kdeI = fitKDE(uI.reshape((-1,1)))
        for j in range(i+1, N):
            jsd = JSD(ds, kdeI, i,j)
            dist[i,j] = jsd
            dist[j,i] = jsd
    return dist

def getKLDMat(ds):
    N = ds.N
    dist = np.zeros((N,N))
    for i in range(N):
        _, uI = getTransformScores(ds,i)
        uI = uI.reshape((-1,1))
        kdeI = fitKDE(uI)
        for j in range(N):
            _,uJ = getTransformScores(ds,j)
            uJ = uJ.reshape((-1,1))
            kdeJ = fitKDE(uJ)
            lnDensI = kdeI.score_samples(uI)
            lnDensJ = kdeJ.score_samples(uI)
            dist[i,j] = KLD(lnDensI, lnDensJ)
    return dist

def getWassersteinMat(ds):
    N = ds.N
    dist = np.zeros((N,N))
    for i in range(N):
        _, uI = getTransformScores(ds,i)
#         uI = uI.reshape((-1,1))
        for j in range(N):
            _,uJ = getTransformScores(ds,j)
#             uJ = uJ.reshape((-1,1))
            dist[i,j] = ss.wasserstein_distance(uI,uJ)
    return dist

def getOptimalAdjacency(trueAlphas):
    N = trueAlphas.shape[0]
    adj = np.zeros((N,N))
    for i,a0 in enumerate(trueAlphas):
        for j,a1 in enumerate(trueAlphas[i+1:],start=i+1):
            adj[i,j] = np.abs(a0 - a1)
            adj[j,i] = np.abs(a0 - a1)
    return adj

In [ ]:
# hide
primeAEs = []
localAEs = []
baselineAEs = []

In [ ]:
# hide
for rep in tqdm(range(10),total=10,leave=False, desc="dataset rep"):
    ds = buildDataset(100,alphaDistr=lambda: np.random.uniform(0.01,0.5))
    ds = addTransformScores(ds)
    # compute dist mat
    wassMat = getWassersteinMat(ds)
    order = np.argsort(ds.trueAlphas.flatten())
    plt.figure()
    sns.heatmap(wassMat[order][:,order])
    plt.title("Wasserstein Matrix orderd by True Alpha")
    plt.show()
    
    plt.figure()
    sns.heatmap(getOptimalAdjacency(ds.trueAlphas)[order][:,order])
    plt.title("Alpha MAE ordered by true alpha")
    plt.show()
    # find partition using Louvain alg
    g = nx.from_numpy_array(1 / np.exp(wassMat))
    partition = community_louvain.best_partition(g)
    values = np.array(list(partition.values()))
    plt.figure()
    for k in np.unique(values):
        print("k:{}    n={}".format(k, (values == k).sum()))
    clusterorder = np.argsort(values)
    sns.heatmap(getOptimalAdjacency(ds.trueAlphas)[clusterorder][:,clusterorder])
    plt.title("Alpha MAE ordered by cluster assignment (K={})".format(len(np.unique(values))))
    plt.show()
    ds.clusters = [np.where(values  == v)[0] for v in np.unique(values)]    
    ds.alphaHats, ds.curves = getBagAlphaHats(ds,10)
    ds.clusterAlphaHat, ds.clusterCurves = getCliqueAlphaHats(ds,ds.clusters, numbootstraps=10)

    ds.alphaPrime = getAlphaPrime(ds.clusters, ds.clusterAlphaHat)

    globalAlphaHat,_ = getCliqueAlphaHats(ds, [np.arange(ds.N)], numbootstraps=10)

    primeAEs.append(np.abs(ds.alphaPrime - ds.trueAlphas))
    localAEs.append(np.abs(ds.alphaHats - ds.trueAlphas))
    baselineAEs.append(np.abs(ds.trueAlphas - globalAlphaHat.mean()))

In [ ]:
# hide
np.mean(primeAEs), np.mean(localAEs), np.mean(baselineAEs)